In [1]:
from modelUnetVGG16 import *
from data import *

vid="v046_5" #version id should match the file number 
# The final number can be [1:5] and denotes the cross-validation order
# Ran DellWS with GeForce RTX3060 GPU

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=45,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

batch_size=3
learning_rate=1e-4
loss=dice_loss

train_gen = trainGenerator(batch_size,'afm_cell_512/train0'+vid[-1],'afmfriction','labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'afm_cell_512/valid0'+vid[-1],'afmfriction','labels',data_gen_args,save_to_dir = None)

# 24 images are used for training, 6 images for validating and 4 images for testing
train_steps = 24//batch_size
valid_steps = 6//batch_size

model = build_vgg16_unet()

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files/unet_cell_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=20*train_steps, validation_steps=valid_steps, 
                    epochs=100, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 24 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Epoch 1/100
160/160 [==============================] - ETA: 0s - loss: 0.2667 - acc: 0.8766 - dice_coef: 0.7333 - iou: 0.5999Found 6 images belonging to 1 classes.
Found 6 images belonging to 1 classes.
160/160 [==============================] - 90s 448ms/step - loss: 0.2667 - acc: 0.8766 - dice_coef: 0.7333 - iou: 0.5999 - val_loss: 0.2818 - val_acc: 0.8649 - val_dice_coef: 0.7182 - val_iou: 0.5629

Epoch 00001: saving model to files\unet_cell_seg_v046_5.hdf5
Epoch 2/100
160/160 [==============================] - 71s 443ms/step - loss: 0.1765 - acc: 0.9306 - dice_coef: 0.8235 - iou: 0.7091 - val_loss: 0.1736 - val_acc: 0.9222 - val_dice_coef: 0.8264 - val_iou: 0.7043

Epoch 00002: saving model to files\unet_cell_seg_v046_5.hdf5
Epoch 3/100
160/160 [==============================] - 71s 444ms/step - loss: 0.1397 - acc: 0.9466 - dice_coef: 0.8603 - iou: 0.7606 - val_loss: 0.1406 - val_acc: 0.9536 - val_dice_

### validate your model and save predicted results

In [3]:
validGene = testGenerator("afm_cell_512/valid0"+vid[-1]+"/predf")
model.load_weights("files/unet_cell_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,6,verbose=1)
saveResult("afm_cell_512/valid0"+vid[-1]+"/predf",results,vid[:4])

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


6/6 [==============================] - 5s 185ms/step
